In [1]:
%load_ext autoreload
%autoreload 2

# Data

In [2]:
from thesis_work.chemberta.utils import load_data_splits

train_df, valid_df, test_df = load_data_splits(protein_type="kinase")

print(train_df.shape, valid_df.shape, test_df.shape)

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/ilker/Documents/MyRepos/thesis-work/.venv/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


(53047, 2) (6631, 2) (6631, 2)


## KFold

In [2]:
from thesis_work.cv.split import create_folds, get_kfold_experiment_indices


length = 50
fold_list = create_folds(length=length)
fold_list

[[17, 8, 34, 3, 31, 12, 28, 20, 2],
 [15, 1, 41, 49, 47, 40, 36, 22, 43],
 [14, 0, 24, 16, 45, 9, 27, 38],
 [46, 5, 39, 21, 32, 37, 29, 4],
 [19, 23, 30, 25, 13, 42, 26, 10],
 [7, 11, 18, 44, 48, 33, 35, 6]]

In [12]:
experiment_indices = get_kfold_experiment_indices(length=length)

for experiment_index in experiment_indices:
    train_df, valid_df, test_df = experiment_index
    print(train_df, valid_df, test_df)

[15, 1, 41, 49, 47, 40, 36, 22, 43, 14, 0, 24, 16, 45, 9, 27, 38, 46, 5, 39, 21, 32, 37, 29, 4, 19, 23, 30, 25, 13, 42, 26, 10] [17, 8, 34, 3, 31, 12, 28, 20, 2] [7, 11, 18, 44, 48, 33, 35, 6]
[17, 8, 34, 3, 31, 12, 28, 20, 2, 14, 0, 24, 16, 45, 9, 27, 38, 46, 5, 39, 21, 32, 37, 29, 4, 19, 23, 30, 25, 13, 42, 26, 10] [15, 1, 41, 49, 47, 40, 36, 22, 43] [7, 11, 18, 44, 48, 33, 35, 6]
[17, 8, 34, 3, 31, 12, 28, 20, 2, 15, 1, 41, 49, 47, 40, 36, 22, 43, 46, 5, 39, 21, 32, 37, 29, 4, 19, 23, 30, 25, 13, 42, 26, 10] [14, 0, 24, 16, 45, 9, 27, 38] [7, 11, 18, 44, 48, 33, 35, 6]
[17, 8, 34, 3, 31, 12, 28, 20, 2, 15, 1, 41, 49, 47, 40, 36, 22, 43, 14, 0, 24, 16, 45, 9, 27, 38, 19, 23, 30, 25, 13, 42, 26, 10] [46, 5, 39, 21, 32, 37, 29, 4] [7, 11, 18, 44, 48, 33, 35, 6]
[17, 8, 34, 3, 31, 12, 28, 20, 2, 15, 1, 41, 49, 47, 40, 36, 22, 43, 14, 0, 24, 16, 45, 9, 27, 38, 46, 5, 39, 21, 32, 37, 29, 4] [19, 23, 30, 25, 13, 42, 26, 10] [7, 11, 18, 44, 48, 33, 35, 6]


In [9]:
import numpy as np
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=False)

# data = np.arange(50)
data = [item for sublist in fold_list[:5] for item in sublist]
data = np.array(data)

for train, test in kfold.split(data):
    print('train: %s, test: %s' % (data[train], data[test]))

train: [15  1 41 49 47 40 36 22 43 14  0 24 16 45  9 27 38 46  5 39 21 32 37 29
  4 19 23 30 25 13 42 26 10], test: [17  8 34  3 31 12 28 20  2]
train: [17  8 34  3 31 12 28 20  2 14  0 24 16 45  9 27 38 46  5 39 21 32 37 29
  4 19 23 30 25 13 42 26 10], test: [15  1 41 49 47 40 36 22 43]
train: [17  8 34  3 31 12 28 20  2 15  1 41 49 47 40 36 22 43 46  5 39 21 32 37
 29  4 19 23 30 25 13 42 26 10], test: [14  0 24 16 45  9 27 38]
train: [17  8 34  3 31 12 28 20  2 15  1 41 49 47 40 36 22 43 14  0 24 16 45  9
 27 38 19 23 30 25 13 42 26 10], test: [46  5 39 21 32 37 29  4]
train: [17  8 34  3 31 12 28 20  2 15  1 41 49 47 40 36 22 43 14  0 24 16 45  9
 27 38 46  5 39 21 32 37 29  4], test: [19 23 30 25 13 42 26 10]


# Model

## Weight Extraction

- [Stackoverflow method](https://stackoverflow.com/questions/66131399/extracting-layer-output-from-classification-model-of-simpletransformer)


In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from thesis_work.chemberta.utils import get_model


model_type = "DeepChem/ChemBERTa-77M-MLM"
# model: ClassificationModel = get_model(model_type=model_type)

model_args = ClassificationArgs(
    evaluate_each_epoch=True,
    evaluate_during_training_verbose=True,
    no_save=True,
    num_train_epochs=10,
    # overwrite_output_dir=True,
    # auto_weights=True, # NOTE: Not working
    # NOTE: Necessary for training outside of Colab
    use_multiprocessing=False,
    # dataloader_num_workers=0,
    # process_count=1,
    use_multiprocessing_for_evaluation=False,
)

model_args_dict = {
    "output_hidden_states": True
}

model = ClassificationModel(
    model_type="roberta",
    model_name=model_type,
    # args=model_args,
    args=model_args_dict,
    # use_cuda=False,
)

Some weights of the model checkpoint at DeepChem/ChemBERTa-77M-MLM were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.out_proj.weight', 'classifier.out

In [4]:
model.__dict__

{'args': ClassificationArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_betas=(0.9, 0.999), adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=True, evaluate_each_epoch=True, fp16=True, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manu

In [ ]:
# Tensorflow

from transformers import RobertaTokenizer, TFRobertaModel

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaModel.from_pretrained('roberta-base')

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple


In [ ]:
# Pytorch

from transformers import RobertaTokenizer, RobertaForTokenClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForTokenClassification.from_pretrained('roberta-base', return_dict=True)


inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

outputs = model(**inputs)
outputs
## last_layer_features = outputs.hidden_states[-1]

In [2]:
from transformers import RobertaTokenizer, RobertaForTokenClassification, RobertaModel

model_name = "DeepChem/ChemBERTa-77M-MLM"
smiles = "O=C(Cc1cccc2ccccc12)Nc1n[nH]c2ccc(N3CCCS3(=O)=O)cc12" # label: 1

# model = RobertaModel.from_pretrained(model_name)
model = RobertaForTokenClassification.from_pretrained(model_name, return_dict=True)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
inputs = tokenizer(smiles, return_tensors="pt")
outputs = model(**inputs)
outputs


/home/ilker/miniconda3/envs/thesis-work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TokenClassifierOutput(loss=None, logits=tensor([[[ 0.0577,  0.0297],
         [ 0.0127, -0.0241],
         [ 0.1406,  0.0758],
         [ 0.0670,  0.0700],
         [-0.0028,  0.1596],
         [-0.2178, -0.1343],
         [-0.0591,  0.1917],
         [ 0.0468,  0.0645],
         [-0.0216,  0.1594],
         [-0.1484,  0.1124],
         [-0.0207,  0.1587],
         [ 0.0078,  0.1441],
         [-0.0238, -0.0986],
         [-0.0262, -0.1147],
         [-0.1084, -0.0714],
         [-0.0500,  0.0045],
         [ 0.0169,  0.0943],
         [ 0.2645,  0.1887],
         [-0.0382,  0.0958],
         [ 0.0390,  0.0884],
         [ 0.0404,  0.0604],
         [-0.0943, -0.0544],
         [-0.1951,  0.1104],
         [-0.0741, -0.1171],
         [-0.0523, -0.0024],
         [-0.2502, -0.0076],
         [ 0.0089, -0.0436],
         [-0.0857, -0.0307],
         [-0.1610,  0.0350],
         [-0.0249,  0.1610],
         [-0.0710, -0.0180],
         [-0.1207,  0.0441],
         [-0.1839,  0.1096],
   